# 05 - Text Generation

This is an optional notebook for testing the text generation capabilities of gpt2. <br>

## 1. Import Packages <a name="import"></a>

In [1]:
import pandas as pd

import pickle
import os

In [2]:
import tensorflow as tf

C:\Users\chivk\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\chivk\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\chivk\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\chivk\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\pyth

In [3]:
#if you have a gpu, use tensorflow-gpu==1.14 instead of tensorflow
#gpt_2_simple requires tensorflow.contrib
#which only exists in tensorflow versions =<1.14
import gpt_2_simple as gpt2 

from tensorflow.python.client import device_lib

#this is to ensure your gpu is detected
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9407221673825764197
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3146173644
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8670898653399447291
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


## Table of Contents <a name="table"></a>
1. [Import Packages](#import)
2. [Load Data](#load)
3. [Text Generation](#text)
    1. [GPU Version](#gpu)
    2. [CPU Version](#cpu)

## 2. Load Data <a name="load"></a>

In [4]:
movie_id = 'tt0167260'

movie_reviews = pd.read_pickle('./data/processed_movie_reviews.pkl.gz', compression = 'gzip')

labels = pickle.load(open('./data/labels.pkl', 'rb'))

movie_reviews = movie_reviews[labels == 1]

movie_reviews = movie_reviews[movie_reviews['movie_id'] == movie_id]

movie_reviews = movie_reviews

movie_reviews.head()

,movie_id,review_text,rating,review_summary,similarities_lsa,similarities_lda,similarities_nmf,flesch_reading_ease_score,difficult_words_score,linsear_write_score,compound_score
15981,tt0167260,If you want a movie which you can watch many t...,10,my favorite movie of all time,0.638689,0.742431,0.673345,-18,8,56,0.4588
15990,tt0167260,"UM dos Malorie's films DE nos so tempo, a tril...",10,Impressionante,0.642445,0.554692,0.545042,49,35,13,0.0000
15991,tt0167260,This is simply the best movie I have ever seen...,10,Best Movie I've Ever Seen,0.500536,0.750266,0.594067,78,18,11,0.6369
15994,tt0167260,If the first film bored you to tears and the s...,1,Supreme boredom,0.786593,0.656013,0.613355,93,14,6,0.3182
15997,tt0167260,"I loved the first Lord of the Rings, and The T...",2,Could this film take longer to end?,0.677961,0.720137,0.658685,70,19,8,0.0000


In [5]:
movie_reviews['review_text'].to_csv('./data/non_spoilers_LOTR.csv', header = False, index = False)

Return to [Table of Contents](#table)

## 3. Text Generation <a name="text"></a>

In [4]:
model_name = "117M"
model_dir = 'text_generator'


if not os.listdir('text_generator'):
    gpt2.download_gpt2(model_name=model_name, model_dir = model_dir)   

### 3A. GPU Version <a name="gpu"></a>

Currently, the GPU version does not work. <br>
You may need at least 8 GB of memory while I only had 4 GB.

In [13]:
#currently a workaround to reset the graph to continue finetuning
tf.reset_default_graph()

config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allocator_type = 'BFC'
config.gpu_options.per_process_gpu_memory_fraction = 0.95
config.gpu_options.allow_growth=True

with tf.Session(config=config) as sess:
 
    gpt2.finetune(sess,
                  './data/non_spoilers_LOTR.csv',
                  model_name = model_name,
                  model_dir = 'text_generator',
                  steps=1000)   # steps is max number of training steps

### 3B. CPU Version <a name="cpu"></a>

In [ ]:
#currently a workaround to reset the graph to continue finetuning
tf.reset_default_graph()

with tf.Session() as sess:
     with tf.device("/cpu:0"):
 
        gpt2.finetune(sess,
                      './data/non_spoilers_LOTR.csv',
                      model_name = model_name,
                      model_dir = 'text_generator',
                      steps=1000)   # steps is max number of training steps

Loading checkpoint text_generator\117M\model.ckpt


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 41.83it/s]

Loading dataset...


dataset has 153088 tokens
Training...
[1 | 74.45] loss=3.07 avg=3.07
[2 | 139.31] loss=3.39 avg=3.23
[3 | 204.10] loss=3.08 avg=3.18
[4 | 273.63] loss=3.03 avg=3.14
[5 | 339.89] loss=3.20 avg=3.16
[6 | 407.50] loss=3.25 avg=3.17
[7 | 476.06] loss=3.01 avg=3.15
[8 | 544.89] loss=3.26 avg=3.16
[9 | 615.20] loss=3.06 avg=3.15
[10 | 685.64] loss=2.96 avg=3.13
[11 | 756.47] loss=3.27 avg=3.14
[12 | 822.95] loss=3.00 avg=3.13
[13 | 889.80] loss=2.88 avg=3.11
[14 | 955.91] loss=2.64 avg=3.07
[15 | 1021.15] loss=2.96 avg=3.07
[16 | 1089.55] loss=2.91 avg=3.06
[17 | 1159.91] loss=3.08 avg=3.06
[18 | 1230.88] loss=2.86 avg=3.05
[19 | 1302.68] loss=2.99 avg=3.04
[20 | 1372.85] loss=3.05 avg=3.04
[21 | 1443.65] loss=2.99 avg=3.04
[22 | 1510.32] loss=3.19 avg=3.05
[23 | 1580.49] loss=3.17 avg=3.05
[24 | 1648.84] loss=2.93 avg=3.05


In [ ]:
gen_texts = gpt2.generate(sess, return_as_list = True)

Return to [Table of Contents](#table)